# Part 1

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim 
#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

In [50]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
table = BeautifulSoup(page.text, 'html.parser').find_all('table') # webscrapping using beautifulsoup
df = pd.read_html(str(table))[0] #change to a pandas dataframe

In [64]:
df_toronto = df[df['Borough'] != 'Not assigned'].reset_index(drop=True) #drop borough that is Not assigned.
print (df_toronto.shape)
print (df_toronto.head())

(103, 3)
  Postal code           Borough                                  Neighborhood
0         M3A        North York                                     Parkwoods
1         M4A        North York                              Victoria Village
2         M5A  Downtown Toronto                    Regent Park / Harbourfront
3         M6A        North York             Lawrence Manor / Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government


# Part 2

In [86]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [83]:
import csv
#get latitude and longitude
url2 = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url2)   
print(df_geo.head(10))
print(df_geo.shape)

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
5         M1J  43.744734 -79.239476
6         M1K  43.727929 -79.262029
7         M1L  43.711112 -79.284577
8         M1M  43.716316 -79.239476
9         M1N  43.692657 -79.264848
(103, 3)


In [85]:
#merge two datasets
df_geo.columns = ['Postal code','Latitude','Longitude']
df_total = pd.merge(df_toronto,df_geo, on='Postal code')
print(df_total.head(10))
print(df_total.shape)

  Postal code           Borough                                  Neighborhood  \
0         M3A        North York                                     Parkwoods   
1         M4A        North York                              Victoria Village   
2         M5A  Downtown Toronto                    Regent Park / Harbourfront   
3         M6A        North York             Lawrence Manor / Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   
5         M9A         Etobicoke                              Islington Avenue   
6         M1B       Scarborough                               Malvern / Rouge   
7         M3B        North York                                     Don Mills   
8         M4B         East York              Parkview Hill / Woodbine Gardens   
9         M5B  Downtown Toronto                      Garden District, Ryerson   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  4

In [96]:
#create map of toronto
latitude = 43.6532
longitude =  -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

#add neighborhoods
for lat, lng, borough, neighborhood in zip(df_total['Latitude'], df_total['Longitude'], df_total['Borough'], df_total['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto